__собираем предложения о земельных участках__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/land/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_land.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/land/avito/avito_2022-09-08_16-05_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-12_13-51_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-13_13-56_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-15_10-31_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-16_15-10_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-19_12-39_raw.xlsx']

In [7]:
# for f in raw_data_files:
#     df = pd.read_excel(f)
#     df['place']='sevastopol'
#     df.to_excel(f,index=False)
#     print(f)

In [8]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

6619


In [9]:
# data_avito

In [10]:
from lib.avito import AvitoDataCleanerRealtyLand

data_avito = AvitoDataCleanerRealtyLand().transform( data_avito )

print(len(data_avito))

6619


In [11]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,area,is_IJS,priceM,priceMU,area_size_category
741,1194440696,"Участок 16,5 сот. (ИЖС)",48000000,,"Гагаринский муниципальный округ, наб. Андрея Первозванного, 7","Продается видовой, ровный участок 16,5 сот. В Севастополе на набережной Андрея Первозванного — лучший коттеджный поселок в городе. \n\nРасположен участок на берегу моря, с красивым видом на море и предназначен для строительства индивидуального жилого дома. \n",16,2022-09-08 16:04:40.227,sevastopol,16.50,True,48.00,2.91,8-20
2304,2244803485,Участок 4 сот. (ИЖС),1800000,,"садоводческое товарищество Слип, 211","Продам ровный, ухоженный в обжитом кооперативе участок, из коммуникаций свет 5 квт можно через тех условия получить 15 квт вода кооперативная, участок находится возле шикарного леса, подъезд к СТ асфальтированный планируется запуск маршрутки, отличное мест",3,2022-09-13 13:56:54.600,sevastopol,4.00,True,1.80,0.45,2-4
6600,2523399183,Участок 5 сот. (ИЖС),2500000,,Фёдоровская ул.,"Срочно продам! Земельный участок в Любимовке. Ижс, 5 сот. \n\nАктивно развивающийся посёлок со всей инфраструктурой и транспортом. Рядом пляж (2,5 км), лес. Документы РФ. \n\nЗвоните я с удовольствием отвечу на ваши вопросы!",22,2022-09-19 12:39:53.571,sevastopol,5.00,True,2.50,0.50,4-8


In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6619 entries, 0 to 6618
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            6619 non-null   int64         
 1   title               6619 non-null   object        
 2   price               6619 non-null   int64         
 3   obj_name            6619 non-null   object        
 4   adr                 6619 non-null   object        
 5   description         6619 non-null   object        
 6   avito_page          6619 non-null   int64         
 7   ts                  6619 non-null   datetime64[ns]
 8   place               6619 non-null   object        
 9   area                6619 non-null   float64       
 10  is_IJS              6619 non-null   bool          
 11  priceM              6619 non-null   float64       
 12  priceMU             6619 non-null   float64       
 13  area_size_category  6619 non-null   category    

---

In [14]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

6619


### обновляем таблицу адресов

In [15]:
from lib.locator import AddressTransformer

data['adr'] = AddressTransformer().transform( data['adr'] )

places = { # поле 'place' и название города
#'bahchisaray': 'Бахчисарай',
#      'sudak': 'Судак',
 'sevastopol': 'Севастополь,',
}

for k,v in places.items():
    print(k,v) # докидываем в строку адреса название города если его нет
    f =  (data['place']==k) & (~data['adr'].str.match(v))
    data.loc[f,'adr'] = v +' '+ data.loc[f,'adr']

sevastopol Севастополь,


---

In [16]:
from lib.locator import LocationUpdater
# from lib.locator import AddressTransformerSev
from lib.locator import GeocoderYandex
from yandex_keys import keys


loc_file_path = 'data/location.pkl'# таблица адресов

loc = LocationUpdater(
        locator=GeocoderYandex(keys[2]),
        # address_transformer=AddressTransformerSev(),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-19 12:41:32 | LocationUpdater: 10172 addresses in location table
[INFO    ] 2022-09-19 12:41:32 | LocationUpdater: 10187 addresses total
[INFO    ] 2022-09-19 12:41:32 | LocationUpdater: 10172 addresses defined
[INFO    ] 2022-09-19 12:41:32 | LocationUpdater: 15 addresses undefined


  0%|          | 0/15 [00:00<?, ?it/s]

[INFO    ] 2022-09-19 12:41:37 | LocationUpdater: 15 new addresses found


In [17]:
loc.sample(3)

,adr,latitude,longitude,truncated
1462,"Севастополь, улица Куликово Поле, 7",44.58,33.52,False
29,"Севастополь, улица Сенявина, 5",44.58,33.52,False
1002,"Севастополь, улица Героев Севастополя, 14",44.61,33.54,False


In [18]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

### дополняем данные геометкой

In [19]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

всего записей: 6636
записей без геометки: 0


In [20]:
assert len(data)>0
data.to_pickle(result_file_path)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6636 entries, 0 to 6635
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            6636 non-null   int64         
 1   title               6636 non-null   object        
 2   price               6636 non-null   int64         
 3   obj_name            6636 non-null   object        
 4   adr                 6636 non-null   object        
 5   description         6636 non-null   object        
 6   avito_page          6636 non-null   int64         
 7   ts                  6636 non-null   datetime64[ns]
 8   place               6636 non-null   object        
 9   area                6636 non-null   float64       
 10  is_IJS              6636 non-null   bool          
 11  priceM              6636 non-null   float64       
 12  priceMU             6636 non-null   float64       
 13  area_size_category  6636 non-null   category    